In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta

import ProcessData

C:\Users\nms198\OneDrive - Newcastle University\3_Coding\Git\ToOLTuBESv3\Scripts\ProcessData.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def setupIOFolder(folder):
    scripts_dir = folder / "Scripts"
    if not os.path.exists(folder):
        os.mkdir(folder)
        print("Created Project folder: folder.name")
    else: print("Using already existing Project folder (" + folder.name + ")")
    if not os.path.exists(scripts_dir):
        os.mkdir(scripts_dir)
        print("Created Scripts folder")
    else: print("Using already existing Scripts folder")
    sys.path.append(str(scripts_dir))
    
    cdi_path = scripts_dir / "CustomDataImports.py"
    if not os.path.exists(cdi_path):
        with open(cdi_path, 'w') as f:
            f.write('''\
from config import *
from pathlib import Path
import pandas as pd

config['filetypes'] = ['xls', 'csv', 'txt']
def fileImport(dataset, folder, filename, pat):
    data_folder_path = Path(config['info']['datasets'].query('dataset == "' + dataset + '" & folder == "' + str(folder) + '"')['data_folder_path'][dataset])
    skiprows = config['info']['datasets'].query('dataset == "' + dataset + '" & folder == "' + str(folder) + '"')['skiprows'][dataset]
    if pat == config['filetypes'][0]:
        df = pd.read_excel(data_folder_path / filename, skiprows=skiprows)
    elif pat == config['filetypes'][1]:
        df = pd.read_csv(data_folder_path / filename, skiprows=skiprows)
    elif pat == config['filetypes'][2]:
        df = pd.read_csv(data_folder_path / filename, sep="\t", skiprows=skiprows)
    else:
        raise ValueError("Unknown file type!")
    return df

### Insert pre-import functions here: names as -> get_DATASET_support_data()

###

# Set pre-import functions inside dictionary separated by commas as -> 'DATASET': get_DATASET_support_data

preimport_functions = {}

### Insert data import functions here: names as -> mod_imported_DATASET_data()

###

# Set data import functions inside dictionary separated by commas as -> 'DATASET': mod_imported_DATASET_data

import_functions = {}                 
''')
        print("Created CustomDataImports script placeholder")
    else: print("Using already existing CustomDataImports script")

## Set analysis folder and project name

C:\Users\nms198\OneDrive - Newcastle University\3_Coding\Python

Rod BES BEWISE

In [3]:
setup_dict = {}

#path = Path(input("Enter the path to create your analysis folder in: "))
path = Path("C:\\Users\\nms198\\OneDrive - Newcastle University\\3_Coding\\Python")
#setup_dict['project'] = input("Enter the name of this project: ")
setup_dict['project'] = "Rod BES BEWISE"
folder = path / setup_dict['project'].replace(" ", "_")
setupIOFolder(folder)

Using already existing Project folder (Rod_BES_BEWISE)
Using already existing Scripts folder
Using already existing CustomDataImports script


## Setup

01/12/2017 13:15:00

14/01/2019 10:27:00

4

In [4]:
#setup_dict['date_start_utc'] = ProcessData.setUTCDatetime(input("Enter the START datetime in UTC timezone (format dd/mm/yyyy  hh:mm:ss): "), "UTC")
#setup_dict['date_end_utc'] = ProcessData.setUTCDatetime(input("Enter the END datetime in UTC timezone (format dd/mm/yyyy  hh:mm:ss) [leave blank if ongoing experiment]: "), "UTC")
#setup_dict['refresh_hours'] = int(input("Enter after how many hours data should be refreshed when processing data: "))

setup_dict['date_start_utc'] = ProcessData.setUTCDatetime("01/12/2017 13:15:00", "UTC")
setup_dict['date_end_utc'] = ProcessData.setUTCDatetime("14/01/2019 10:27:00", "UTC")
setup_dict['refresh_hours'] = int(4)

In [5]:
setup_df = pd.DataFrame(setup_dict.items(), columns = ['id','value'])

In [6]:
for row in range(0,len(setup_df)):
    if isinstance(setup_df.loc[row,'value'], datetime):
        setup_df.loc[row,'value'] = setup_df.loc[row,'value'].replace(tzinfo=None)
setup_df

,id,value
0,project,Rod BES BEWISE
1,date_start_utc,2017-12-01 13:15:00
2,date_end_utc,2019-01-14 10:27:00
3,refresh_hours,4


In [7]:
with pd.ExcelWriter(folder / 'Info_test.xlsx',
                        engine = 'xlsxwriter',
                        datetime_format = 'dd/mm/yyyy  hh:mm:ss') as writer:
    setup_df.to_excel(writer, sheet_name='setup', index = False)

In [8]:
dataset	folder	selection	melt	data_folder_path	supp_data_filepath	file_pat	skiprows	Slim	Del_unit_rows	Add_blank_rows


WindowsPath('C:/Users/nms198/OneDrive - Newcastle University/3_Coding/Python/Rod_BES_BEWISE/Info_test.xlsx')

In [16]:
df = pd.DataFrame(columns=["dataset", "folder", "selection", "melt", "data_folder_path", "supp_data_filepath", "file_pat", "skiprows", "Slim", "Del_unit_rows", "Add_blank_rows"])
parts = int(input("Enter the number of dataset folders to import:"))

for _ in range(parts):
    dataset = input("Enter DATASET name:")
    folder = int(input(dataset + ": Enter FOLDER number:"))
    selection = input(dataset + ": Enter SELECTION name:")
    melt = input(dataset + ": Enter MELT name:")
    data_folder_path = Path(input(dataset + ": Enter DATA FOLDER PATH name:"))
    supp_data_filepath = Path(input(dataset + ": Enter SUPPORTING DATA FILEPATH name:"))
    file_pat = input(dataset + ": Enter FILE PATTERN as .***:")
    skiprows = input(dataset + ": Enter number of top rows to SKIP in datafile:")
    Slim = input(dataset + ": Enter 1 for SLIM or leave blank for wide datafile:")
    Del_unit_rows = input(dataset + ": Enter 1 to DELETE UNIT ROWS after headers or leave blank for no unit deletion in datafile:")
    Add_blank_rows = input(dataset + ": Enter 1 to ADD BLANK ROWS after datafile or leave blank for continuous data:")
    
    df1 = pd.DataFrame(data=[[dataset,folder,selection,melt,data_folder_path,supp_data_filepath,file_pat,skiprows,Slim,Del_unit_rows,Add_blank_rows]],
                       columns=["dataset", "folder", "selection", "melt", "data_folder_path", "supp_data_filepath", "file_pat", "skiprows", "Slim", "Del_unit_rows", "Add_blank_rows"])
    df = pd.concat([df,df1], axis=0)

df.index = range(len(df.index))
df

Enter the number of dataset folders to import:0


,dataset,folder,selection,melt,data_folder_path,supp_data_filepath,file_pat,skiprows,Slim,Del_unit_rows,Add_blank_rows


In [22]:
def addDataset(df):
    dataset = input("Enter DATASET name:")
    folder = int(input(dataset + ": Enter FOLDER number:"))
    selection = input(dataset + ": Enter SELECTION name:")
    melt = input(dataset + ": Enter MELT name:")
    data_folder_path = Path(input(dataset + ": Enter DATA FOLDER PATH name:"))
    supp_data_filepath = Path(input(dataset + ": Enter SUPPORTING DATA FILEPATH name:"))
    file_pat = input(dataset + ": Enter FILE PATTERN as .***:")
    skiprows = input(dataset + ": Enter number of top rows to SKIP in datafile:")
    Slim = input(dataset + ": Enter 1 for SLIM or leave blank for wide datafile:")
    Del_unit_rows = input(dataset + ": Enter 1 to DELETE UNIT ROWS after headers or leave blank for no unit deletion in datafile:")
    Add_blank_rows = input(dataset + ": Enter 1 to ADD BLANK ROWS after datafile or leave blank for continuous data:")
    
    df1 = pd.DataFrame(data=[[dataset,folder,selection,melt,data_folder_path,supp_data_filepath,file_pat,skiprows,Slim,Del_unit_rows,Add_blank_rows]],
                       columns=["dataset", "folder", "selection", "melt", "data_folder_path", "supp_data_filepath", "file_pat", "skiprows", "Slim", "Del_unit_rows", "Add_blank_rows"])
    df = pd.concat([df,df1], axis=0)
    df.index = range(len(df.index))
    return(df)

In [23]:
df = pd.DataFrame(columns=["dataset", "folder", "selection", "melt", "data_folder_path", "supp_data_filepath", "file_pat", "skiprows", "Slim", "Del_unit_rows", "Add_blank_rows"])


In [25]:
df = addDataset(df)
df

Enter DATASET name:abc
abc: Enter FOLDER number:1
abc: Enter SELECTION name:1
abc: Enter MELT name:1
abc: Enter DATA FOLDER PATH name:1
abc: Enter SUPPORTING DATA FILEPATH name:1
abc: Enter FILE PATTERN as .***:1
abc: Enter number of top rows to SKIP in datafile:1
abc: Enter 1 for SLIM or leave blank for wide datafile:1
abc: Enter 1 to DELETE UNIT ROWS after headers or leave blank for no unit deletion in datafile:1
abc: Enter 1 to ADD BLANK ROWS after datafile or leave blank for continuous data:1


,dataset,folder,selection,melt,data_folder_path,supp_data_filepath,file_pat,skiprows,Slim,Del_unit_rows,Add_blank_rows
0,abc,1,1,1,1,1,1,1,1,1,1
1,abc,1,1,1,1,1,1,1,1,1,1
